In [32]:
tickers = ['PETR4.SA' , 'BBAS3.SA'] #definimos os tickers das ações
periodo = '5y'



In [42]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go 
import yfinance as yf

def fix_columns(df):
    return ['IBOV' if col == '^BVSP' else col.rstrip('.SA') for col in df.columns]


prices = yf.download(tickers=tickers , period=periodo, rounding  = True)['Adj Close']
prices.columns = fix_columns(prices)
prices.dropna(inplace=True)
retorno = prices.pct_change().dropna()
ativos = retorno.columns.to_list()

retorno


[*********************100%%**********************]  2 of 2 completed


,BBAS3,PETR4
Date,,
2019-06-10,-0.009307,-0.004197
2019-06-11,0.019833,0.018967
2019-06-12,-0.008700,-0.011375
2019-06-13,-0.016004,0.012552
2019-06-14,-0.017314,-0.005165
...,...,...
2024-06-03,0.001106,-0.005414
2024-06-04,0.002947,-0.011146
2024-06-05,0.002203,0.001311


In [69]:
prices

retorno.

2.876180482686254

#### 1. Criando a carteira

#### Cálculo da variância da carteira: com 2 ativos

$$ \sigma_{p}² = \sigma_{1}^{2} w_1^2 + \sigma_{2}^{2} w_2^2 + 2 \sigma_1 \sigma_2 w_1 w_2 \rho_{12}$$

In [65]:
import numpy as np
import pandas as pd

# Dados de retorno diário das ações BBAS3 e PETR4
data = retorno


# Função para calcular o retorno e risco da carteira
def carteira_calculo(df, weights):
    carteira_list = []
    correlation = df.corr()
    rho_12 = correlation.iloc[0, 1]
    std_dev = df.std()
    sigma_1, sigma_2 = std_dev.iloc[0], std_dev.iloc[1]
    mu_1, mu_2 = df.mean().iloc[0], df.mean().iloc[1]

    for w_1 in weights:
        w_2 = 1 - w_1
        retorno = (mu_1 * w_1 + mu_2 * w_2) * 252
        risco = np.sqrt((sigma_1**2 * w_1**2 + sigma_2**2 * w_2**2 + 2 * rho_12 * w_1 * w_2 * sigma_1 * sigma_2) * 252)
        carteira_list.append({'retorno': retorno, 'risco': risco, 'pesos': w_1})
    
    carteira = pd.DataFrame(carteira_list)
    return carteira

# Pesos para a carteira
weights = np.linspace(start=0, stop=1, num=200)

# Calculando a carteira
carteira = carteira_calculo(df, weights)

carteira['retorno(%)'] = carteira['retorno'] * 100
carteira['risco(%)'] = carteira['risco'] * 100

# Plotando o gráfico do retorno vs risco da carteira
fig = px.scatter(
    carteira,
    x='risco(%)',
    y='retorno(%)',
    color='pesos',
    title='Retorno vs Risco da Carteira',
    labels={'risco(%)': 'Risco (%)', 'retorno(%)': 'Retorno (%)', 'color': 'Risco (%)'},
    hover_data={'pesos': True, 'risco(%)': ':.2f', 'retorno(%)': ':.2f'}
)

# Adicionando a legenda personalizada
fig.update_layout(
    legend_title_text='BBAS3'
)

fig.show()